# <span style="color:#84b6f4;">Detección de comentarios de hate</span>

### <span style="color:#77dd77;">Formación del DataFrame</span>

En primer lugar, sacamos los datos del fichero txt y formamos un DataFrame con ellos:

In [1]:
import pandas as pd
import os

# Función para leer y transformar un archivo Parquet.
def read_and_transform(parquet_path):
    df = pd.read_parquet(parquet_path)
    columns_of_interest = [col for col in df.columns if col == 'text' or col == 'has_threat']
    df = df[columns_of_interest]
    return df

# Especificamos los archivos Parquet utilizando os.path.join.
parquet_files = ['0000.parquet', '0001.parquet', '0002.parquet']
data_frames = []

for file_name in parquet_files:
    file_path = os.path.join(os.getcwd(), 'resources', file_name)
    data_frames.append(read_and_transform(file_path))

# Concatenamos todos los DataFrames.
concatenated_data_frame = pd.concat(data_frames, ignore_index=True)

# Mostramos las primeras filas del DataFrame balanceado.
print(concatenated_data_frame.head(5))

                                                text  has_threat
0  Un poco solitario aquí no parece tener muchos ...           0
1  @ManuelViloria ¡Gracias!Estoy un poco asustado...           0
2  txt chat con Jake lmfao it frikkinawesomei ech...           0
3  Soy un estudiante de primer año en la universi...           0
4  espera montón de videos aprender idiomas sueño...           0


### <span style="color:#77dd77;">Entrenamiento</span>

Seleccionamos las métricas que usaremos para predecir, así como el atributo objetivo que, en este caso, será numérico.

In [2]:
# Filtramos el DataFrame para obtener las muestras con has_threat = 1
threat_df = concatenated_data_frame[concatenated_data_frame['has_threat'] == 1]

# Filtramos el DataFrame para obtener las muestras con has_threat = 0
non_threat_df = concatenated_data_frame[concatenated_data_frame['has_threat'] == 0].sample(n=1000, random_state=42)

# Concatenamos los DataFrames balanceados
balanced_df = pd.concat([threat_df, non_threat_df], ignore_index=True)

# Mezclamos las muestras para asegurar un orden aleatorio
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Atributo que usaremos para predecir
text = balanced_df['text']

# Atributo objetivo a predecir
goal = balanced_df['has_threat']

A continuación, entrenamos un modelo de clasificación con una red neuronal:

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.feature_extraction.text import CountVectorizer

# Dividir los datos en conjuntos de entrenamiento (70%) y prueba (30%)
attributes_train, attributes_test, goal_train, goal_test = train_test_split(text, goal, test_size=0.3, random_state=42)

# Creamos un vectorizador de texto
vectorizer = CountVectorizer()
attributes_train_vectorized = vectorizer.fit_transform(attributes_train)
attributes_test_vectorized = vectorizer.transform(attributes_test)

# Entrenamos el modelo de clasificación de redes neuronales
threat_classifier = MLPClassifier(hidden_layer_sizes=(50, 50, 50), activation='relu', solver='adam', random_state=42, max_iter=1000)
threat_classifier.fit(attributes_train_vectorized, goal_train)

# Realizamos predicciones con el conjunto de prueba
prediction = threat_classifier.predict(attributes_test_vectorized)

# Calculamos la precisión y F1-Score del modelo
accuracy = accuracy_score(goal_test, prediction)
f1 = f1_score(goal_test, prediction)
print("Precisión del modelo en el conjunto de prueba:", accuracy)
print("F1-Score del modelo:", f1)

# Mostramos la matriz de confusión
print(confusion_matrix(goal_test, prediction))

Precisión del modelo en el conjunto de prueba: 0.7261698440207972
F1-Score del modelo: 0.7322033898305085
[[203  98]
 [ 60 216]]


Probamos el modelo con algunos ejemplos concretos:

In [4]:
message = ["NO es una amenaza", "es una amenaza"]

# Texto de ejemplo 1
example_text_1 = "Me encanta la peli."
example_text_1_vectorized = vectorizer.transform([example_text_1])

# Texto de ejemplo 2
example_text_2 = "Iremos a buscarte"
example_text_2_vectorized = vectorizer.transform([example_text_2])

# Texto de ejemplo 3
example_text_3 = "Te felicito por tu trabajo; sin embargo, tienes que mejorar tu oratoria."
example_text_3_vectorized = vectorizer.transform([example_text_3])

# Texto de ejemplo 4
example_text_4 = "Te voy a matar."
example_text_4_vectorized = vectorizer.transform([example_text_4])

# Texto de ejemplo 5
example_text_5 = "Te quiero, pero eres una basura."
example_text_5_vectorized = vectorizer.transform([example_text_5])

print("El comentario '" + example_text_1 + "' " + message[threat_classifier.predict(example_text_1_vectorized)[0]] + ".")
print("El comentario '" + example_text_2 + "' " + message[threat_classifier.predict(example_text_2_vectorized)[0]] + ".")
print("El comentario '" + example_text_3 + "' " + message[threat_classifier.predict(example_text_3_vectorized)[0]] + ".")
print("El comentario '" + example_text_4 + "' " + message[threat_classifier.predict(example_text_4_vectorized)[0]] + ".")
print("El comentario '" + example_text_5 + "' " + message[threat_classifier.predict(example_text_5_vectorized)[0]] + ".")

El comentario 'Me encanta la peli.' es una amenaza.
El comentario 'Iremos a buscarte' es una amenaza.
El comentario 'Te felicito por tu trabajo; sin embargo, tienes que mejorar tu oratoria.' NO es una amenaza.
El comentario 'Te voy a matar.' es una amenaza.
El comentario 'Te quiero, pero eres una basura.' NO es una amenaza.


Para finalizar, exportamos el modelo:

In [5]:
from joblib import dump

# Guardamos el modelo
model_filename = 'threat_classifier.joblib'
dump(threat_classifier, model_filename)

# Guardamos el vectorizador
vectorizer_filename = 'threat_vectorizer.joblib'
dump(vectorizer, vectorizer_filename)

['threat_vectorizer.joblib']